### Libraries import

In [1]:
import pandas as pd # Dataframes management
from zipfile import ZipFile  # Files compressed management
import os # Files management along OS
import re

In [2]:
from os import mkdir
from os.path import exists

In [3]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import Options

In [4]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [5]:
import logging
import datetime
import sys

### Modules

In [6]:
from db import *

### Data import

In [7]:
# Specifying the name of the zip file
fileZIP = "/items_ordered_2years_V2.zip"
fileCSV = "/items_ordered_2years_V2.csv"

path = "../Inputs/Modificados - Atmira_Pharma_Visualization"
  
# Open the zip file in read mode
with ZipFile(f"{path}{fileZIP}", 'r') as zip: 
    # List all the contents of the zip file
    zip.printdir() 
  
    # Extract all files
    print('extraction...') 
    zip.extractall(path) 
    print('Done!')

#Import CSV to pandas
itemsOrdered = pd.read_csv(f"{path}{fileCSV}")
print("CSV imported to Pandas successfully")

# Remove uncompressed CSV file
os.remove(f"{path}{fileCSV}")
print("Original CSV removed to preserve repo health")

File Name                                             Modified             Size
items_ordered_2years_V2.csv                    2022-01-10 08:27:40    167020440
extraction...
Done!
CSV imported to Pandas successfully
Original CSV removed to preserve repo health


In [8]:
itemsOrdered.head(3)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,ce30c2f02458457e3c7b563a636ae2a1,0916c05c5c3f65f59d813a78ac35c8d2,2018-11-06 16:52:13,86434,1.0,9.3750,15.38,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
1,ce30c2f02458457e3c7b563a636ae2a1,ff323b39ae36843396d2e53ce549fb10,2018-11-06 16:52:13,87652,1.0,7.0000,15.41,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
2,ce30c2f02458457e3c7b563a636ae2a1,199916dffc95259f4d2daab6664ca9c0,2018-11-06 16:52:13,2785,1.0,9.1587,12.23,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100


In [9]:
itemsOrdered.shape

(930960, 11)

In [11]:
del zip

### Arreglos para facilitar el webscrapping

In [12]:
%%capture
'''
Temas Portugueses
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("4575-049", "4575-001")
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("1750", "1750-001")
'''

In [13]:

itemsOrdered.loc[itemsOrdered['zipcode'].eq('30139') & itemsOrdered['city'].eq('Murcia'), "city"] = "EL RAAL"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("29039", "28039")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33195') & itemsOrdered['city'].eq('Oviedo '), "city"] = "Morente"
itemsOrdered["city"].replace(to_replace={'Cangas Del Narcea':"Cerezaliz"}, inplace=True)
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("O88oo", "08800")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('43890') & itemsOrdered['city'].eq('TARRAGONA'), "city"] = "Hospitalet"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('07700') & itemsOrdered['city'].eq('MAHON'), "city"] = "Grau"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("47021", "46021")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33405'), "city"] = "Raices Nuevo"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('29720') & itemsOrdered['city'].eq('MALAGA'), "city"] = "La Arana"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('08222'), "city"] = "Terrassa"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('03195'), "city"] = "Los Arenales Del Sol"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('08780'), "city"] = "Palleja"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33405'), "city"] = "Raices Nuevo"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("347007", "37007")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('36194'), "city"] = "Perdecanai"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('29631'), "city"] = "Arroyo De La Miel"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('27810'), "city"] = "Sancobade"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("-39840", "39840")

itemsOrdered.loc[itemsOrdered['zipcode'].eq('50620'), "city"] = "Casetas"

itemsOrdered.loc[itemsOrdered['zipcode'].eq('50620'), "city"] = "Casetas"

### USO DE SCRAPPING PARA CÓDIGO POSTAL

Creación de Base de Datos para almacenamiento de resultados formateados del Scrapping

In [15]:
def IntroDB():
    try:
        pathDB = "../databases/"
        nameDB= "scrapedZips.db"
        if exists(pathDB):
            print(f"Carpeta {pathDB} encontrada")
        else:
            mkdir(pathDB)
            print (f"Creada carpeta {pathDB}")
    except OSError:
        print (f"Creación de carpeta {pathDB} falló")
        
    if exists(f"{pathDB}{nameDB}"):
        print("Database existe")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        return con, cur
    else:
        print("Database no existe")
        CreateCon(f"{pathDB}{nameDB}")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        PrepareCon(con, cur, option="insert",
            values=("Test","Test","Test","Test","Test"))
        return con, cur


In [16]:
con, cur = IntroDB()

Carpeta ../databases/ encontrada
Database existe
Conexión establecida


In [17]:
%%capture
try:
    if GetThings(cur, selection="id_ori,Country,Region,City,Zipcode", where=["ID", 1], limit=1)[0][0] == "Test":
        PrepareCon(con,cur,where=["ID",1],option="delete")
        cur.execute("UPDATE `sqlite_sequence` SET `seq` = 0 WHERE `name` = 'ZipsInfo';")
except IndexError:
    pass

'''
Check the next query if something perturbe your mind
SELECT * FROM `sqlite_sequence`;
'''

In [18]:
GetThings(cur, selection="Country,Region,City,Zipcode", where=["ID", 1], limit=1)

[('Spain', 'Comunidad Valenciana', 'Valencia', '46019')]

A continuación se genera una lista compuesta de tuplas compuestas de la siguiente forma: ("Ciudad", "Zipcode")

In [33]:
rawDataZipcode = list(zip(itemsOrdered["city"].tolist(), itemsOrdered["zipcode"].tolist()))
# rawDataZipcode = rawDataZipcode[:int(len(rawDataZipcode)/30)]

In [34]:
len(rawDataZipcode)

930960

**Funciones destacadas**

Función que "limpia" los nombres de ciudades para mejorar su emparejamiento automático

In [29]:
def CityCleaner(text):
    stopWordSpanish = set(stopwords.words('spanish'))
    wordTokens = word_tokenize(AcentosLimpiador(text.lower()).rstrip()) 
    filteredSentence = [element for element in wordTokens if not element in stopWordSpanish] 
    return filteredSentence

Función que limpia zipcodes

In [30]:
#Limpieza de zipcodes con RegEx
def num_guion(string):
    """ Get a string with the numbers and hyphens of another string
    
    Args:
        df: string used to extract the string with numbers abd hyphens

    Returns:
        df: the string with numbers and hyphens
    """
    aux = re.match("([\d-]+)", str(string))
    try:
        return str(aux.group())
    except:
        return string


itemsOrdered["zipcode"] = itemsOrdered["zipcode"].apply(lambda x: num_guion(x))

Función que limpia los acentos con el fin de homogeneizar

In [31]:
def AcentosLimpiador(text):
	acentos = {'ñ':'n','á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'à':'a'}
	for ele in acentos:
		if ele in text:
			text = text.replace(ele, acentos[ele])
	return text

Configuración de logging del scrapeo

In [32]:
logger = logging.getLogger('ScrapLog')
logger.setLevel(logging.INFO)
# logger.setLevel(logging.ERROR)

timestamp = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
filename=f'Scrapping{timestamp}.log'
formatter = logging.Formatter('[%(asctime)s] %(name)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')

In [33]:
file_handler = logging.FileHandler(filename=filename)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [34]:
logging.basicConfig(
    filename=filename,
    level=logging.INFO,
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
)

Función que formatea los resultados del webscrapping de forma adecuada a los requerimientos necesarios

In [35]:
def zipCodeManipulation(city, zipcode, queryResult="", saved = False):
  
    # This conditional checks if the zipcode info was scrapped and stored succesfully
    if saved == False:

        listTestingZipcode = queryResult.split("\n")
        indexMatchRegex = list(map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(r"[a-z][A-Z0-9]", x)], listTestingZipcode))

        resultScrapClean = []
        for pas,resultScrap in enumerate(listTestingZipcode):
            for pos,ele in enumerate(indexMatchRegex[pas]):
                if len(indexMatchRegex[pas])==2:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                        resultScrapClean.append(txt)
                elif len(indexMatchRegex[pas])==3:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                    elif pos ==2:
                        txt = txt[:ele[0]+3]+","+txt[ele[1]+1:]
                        resultScrapClean.append(txt)

        resultScrapListed = [element.split(",") for element in resultScrapClean]
        resultScrapRearr = [(element[0], element[1], element[-2], element[-1]) for element in resultScrapListed]

        resultZip = []
        for element in resultScrapRearr:
            for element2  in CityCleaner(element[2]):
                if element2 in CityCleaner(city):
                    resultZip.append(element)
                    break
        try:
            resultZip = resultZip[0]
            resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]
        except IndexError:
            resultZip = []
            for element in resultScrapRearr:
                for element2 in CityCleaner(element[2]):
                    for element3 in CityCleaner(city):
                        if element2 .__contains__(element3):
                            resultZip.append(element)
                            break
            try:
                resultZip = resultZip[0]
                resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]
            except IndexError:
                resultZip = []
                for element in resultScrapRearr:
                    for element2 in CityCleaner(element[2]):
                        for city in CityCleaner(element3):
                            if element3.__contains__(element2):
                                resultZip.append(element)
                                break
                if resultZip != []:
                    resultZip = resultZip[0]
                    resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]
                elif resultZip ==[] : #ELIMINAR ESTA LÍNEA PARA PODER VERIFICAR ERRORES
                    resultZip = ["ERROR1","ERROR","ERROR",zipcode]
                    
                    

    elif saved == True:
        resultZip = GetThings(cur, selection="Country,Region,City,Zipcode", where=["Zipcode", zipcode], limit=1, simplify = True)

    if resultZip[3].startswith(" L'"):
        resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]
    
    if len(resultZip[3]) >= 20 and resultZip[0]=="Portugal":
        resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]

    i = (resultZip[3]+resultZip[3]).find(resultZip[3], 1, -1)
    return resultZip if i == -1 else [resultZip[0],resultZip[1],resultZip[2],zipcode]


**Webscrapping!**

In [36]:
# %%capture cap --no-stderr

logger.info("Starting Webscrapping!")

for pos, element in enumerate(rawDataZipcode):
    try:

        if element[1]=="323903":
            
            if pos in [ele[0]-1 for ele in GetThings(cur, selection="ID", where=["Zipcode", element[1].rstrip()])]:
                logger.info(f"SP - Sin procesar |City: {element[0]} | Zipcode: {element[1]} | Orden del dataframe: {pos}")

            else:
                if GetThings(cur, selection="Zipcode", where=["Zipcode", element[1].rstrip()], limit=1, simplify=True) == None:
                    PrepareCon(con, cur, values=["China", "Zhejiang", "Lishui", "323903",pos], option="insert", verbose=False)
                    logger.info(f"SP - Insertado | City: {element[0]} | Zipcode: {element[1]} | Orden del dataframe: {pos}")
    
                else:
                    zipCodeDef = zipCodeManipulation(city=element[0], zipcode=element[1].rstrip(), saved = True)
                    logger.info(f"SP - Ya guardado en BBDD | City: {element[0]} | Zipcode: {element[1]} | Orden del dataframe: {pos}")
                    logger.info(f"Devuelto de query: {zipCodeDef}")

        if GetThings(cur, selection="Zipcode", where=["Zipcode", element[1].rstrip()], limit=1, simplify=True) == None:  
                
                logger.info(f"Para scrapear |Ciudad: {element[0]} | Zipcode: {element[1]} | Orden del dataframe: {pos}")
    
                zipCodeDef = ["ERROR2","ERROR","ERROR",element[1],pos]
                logger.info(f"No Scrapeado: {zipCodeDef}")

        else:
            if pos in [ele[0]-1 for ele in GetThings(cur, selection="ID", where=["Zipcode", element[1].rstrip()])]:
                logger.info(f"Sin procesar |City: {element[0]} |Zipcode: {element[1]} | Orden del dataframe: {pos}")
                continue
            else:
                logger.info(f"Ya guardado en BBDD  | City: {element[0]} | Zipcode: {element[1]} | Orden del dataframe: {pos}")
                zipCodeDef = zipCodeManipulation(city=element[0], zipcode=element[1].rstrip(), saved = True)
                logger.info(f"Devuelto de query: {zipCodeDef}")

        PrepareCon(con, cur, values=[zipCodeDef[0],zipCodeDef[1],zipCodeDef[2],zipCodeDef[3],pos], option="insert", verbose=False)

    except:
        logger.info(f"No encontrado nada en ciudad: {element[0]}, zipcode: {element[1]}")
        logger.info("--------------------------------------------")
        zipCodeDef= ["ERROR3","ERROR","ERROR",element[1]]
        PrepareCon(con, cur, values=[zipCodeDef[0],zipCodeDef[1],zipCodeDef[2],zipCodeDef[3],pos], option="insert", verbose=False)
        continue


driver.close()

**Comprobaciones del scrapping y formateo posterior realizado**

In [23]:
tableMain = "ZipsInfo"

In [45]:
cur.execute(f"SELECT (select count() from {tableMain}) as count FROM {tableMain} LIMIT 1").fetchone()[0]

321671

**Transformación del scrapeo formateado a dataframe de Pandas**

In [47]:
sqlToList = cur.execute(f"SELECT Country,Region,City,Zipcode FROM {tableMain}").fetchall()

In [48]:
df = pd.DataFrame(sqlToList, columns=["Country","Region","City","Zipcode"])
df.head()

,Country,Region,City,Zipcode
0,Spain,Comunidad Valenciana,Valencia,46019
1,Spain,Comunidad Valenciana,Valencia,46019
2,Spain,Comunidad Valenciana,Valencia,46019
3,Spain,Andalucia,Almeria,04009
4,Spain,Andalucia,Almeria,04009


In [49]:
df["Country"].value_counts()

Spain                 297191
ERROR                  18948
Portugal                4421
Mexico                   525
France                   141
Peru                     104
United States             58
Italy                     43
Colombia                  39
Czech Republic            33
Puerto Rico               30
Germany                   25
India                     25
Russian Federation        21
United Kingdom            16
Poland                    14
Belgium                    6
Canada                     5
Argentina                  5
Switzerland                4
China                      4
Denmark                    4
Croatia                    3
Chile                      2
Austria                    2
Algeria                    1
Thailand                   1
Name: Country, dtype: int64

In [50]:
df.shape

(321671, 4)

In [35]:
# Cierre de la base de datos
con.close()

**Exportación del dataframe a csv para su conservación**

In [37]:

df.to_csv('DataScrapped.csv')

In [38]:
df2 = pd.read_csv("../Inputs/Modificados - Atmira_Pharma_Visualization/DataScrapped.csv")

In [39]:
df2.head()

,Unnamed: 0,Country,Region,City,Zipcode
0,0,Spain,Comunidad Valenciana,Valencia,46019
1,1,Spain,Comunidad Valenciana,Valencia,46019
2,2,Spain,Comunidad Valenciana,Valencia,46019
3,3,Spain,Andalucia,Almeria,04009
4,4,Spain,Andalucia,Almeria,04009
